## 음성인식 : Feature representation (Speech_Recognition : Acoustic Model)
---
__참고사항 :__ 
이 글은 성원용 교수님의 "음성인식 및 합성" 강의와 [Deep Learning for Speech Recognition(Adam Coates, Baidu)](https://www.youtube.com/watch?v=g-sndkf7mCs)를 주로 참고하여 정리하였습니다.  
정리글이므로 오류가 있을 수 있으니 github issue 란에 제보해주시면 확인 후 수정하겠습니다.  
감사합니다.
<br \>

### 0. Acoustic Model 개요  
주로 참고한 자료는 다음과 같다.  
[GMM and HMM, Tata Institute of Fundamental Reserach](http://www.iitg.ac.in/samudravijaya/tutorialSlides/gmmHmmTutoChiefWiSSAP09.pdf)  
[ASR Lectures, www.inf.ed.ac.uk](https://www.inf.ed.ac.uk/teaching/courses/asr/2016-17/asr03-hmmgmm-handout.pdf)  
좋은 글 감사합니다(\_ \_).  

<br \>
음성인식기는 아래식에서 W* 를 구하는 것이다.
$$ W^* = \underset{W}{\text{argmax}} \sum_{Q} p(O|Q)p(Q|W)P(W) $$  
Pronounciation Model이 결정되었다면(ex. Monophone) 아래와 같아진다.  
$$ W^* = \underset{W}{\text{argmax}} \sum_{Q} p(O|W)P(W) $$  
여기서 W는 단어의 나열(word sequence), O는 acoustic feature vectors(MFCC 또는 Mel scaled filter bank 결과 값)일 것이다.  
Acoustic Model에 해당하는 것은 p(O|W)으로 W가 주어졌을 때 O일 확률을 구하는 것이다.  

__그럼 p(O|W)는 어떻게 구하는가??__  
문제를 단순하게 하기 위해 hello를 예로 들어보자. 그럼 p(O|W)는 p(O|hello)라고 적을 수 있을 것이다.   
p(O|hello)는 여러 사람이 hello라고 말한 음성을 모은 뒤 그것들의 acoustic feature vectors을 모아 분포로 만들면 된다. 

분포로 만드려고 보니 문제가 생긴다.  
1. __같은 Pronounciation Model의 최소 단위 마저도 값이 다르다.__  
<br \>
_"경우에 따라 hello를 헬로, 헬로우, 할로, 할룽, 할로우, 헐로 등등으로 읽을 수 있다."_  
<br \>
Pronounciation Model을 Monophone이라고 가정했을 때, hello는 [HH HA L OW]라는 phoneme으로 표현될 수 있다고 하자.  
여기서 여러 사람의 hello 음성에서 [HH] 부분만 추출했을 때 과연 acoustic feature vector 값이 같을까?  
물론 다를 것이다.  
앞서 Pronounciation Model 설명에서 말했듯 앞 뒤 단어에 영향을 받고, 또 화자에 따라, 같은 화자라도 상황에 따라 다를 것이기 때문이다.  
이 것을 분포로 만들어서 예측하는 것이 Acoustic Model이다.  
<br \>
2. __phoneme의 배열이 다르다.__  
<br \>
_"어떤 사람은 hello을 엄청 길게 발음할 수도 있고, 어떤 사람을 hello를 엄청 짧게 발음할 수 있다."_  
<br \>
Pronounciation Model을 Monophone이라고 가정했을 때, hello는 [HH HA L OW]라는 phoneme으로 표현될 수 있다고 하자.  
어떤 사람의 hello 발음은 [HH HH HH HH HA L OW]로 일 수 있고, 또 다른 사람의 hello 발음은 [HH HA L L L L L O]일 수 있다.
<br \>
즉 phoneme의 배열(sequence)이 다를 수 있다는 것이다.  
이 것을 결정해주는 것이 다음 장에서 정리할 Language 모델이다.  

### 1. GMM (Gaussian Mixture Model)
Training 음성 샘플이 다양한 만큼 분포도 다채로운 형태로 나타날텐데, 그것을 여러 개의 가우시안 분포가 혼합된 모델로 표현하는 것이 GMM 이다.  
GMM에 대해서 먼저 알아보고, 그 다음 우리의 문제에 GMM을 적용해보자.

- __복습 __
    - __모수 추정(parameter estimation)__  
    사실 우리가 지금하는 일은 training data을 보고 true distribution p(우리 문제에서 p는 p(O|W))를 찾아내보겠다는 시도이다.  
    이 방법에는 모수 추정방식과 비모수 추정방식으로 크게 나눌 수 있다.  
    모수 추정방식은 true distribution p가 사실은 q일 거야 라고 가정하고 q의 모수를 추정해보는 것이다.  
    비모수 추정방식은 그런 가정 없이 추정하는 것이다.  
    <br \>
    우리가 지금 볼 방법은 모수 추정방식인데,  
    q는 어떤 분포도 될 수 있지만 지금은 GMM을 다루고 있기 때문에 q를 Gaussian(or Normal) Distribution이라고 해보자.  
    <br \>
        - __Gaussian(or Normal) Distribution__  
$$ p(x|\boldsymbol{\theta}) = \frac{1}{\sqrt{2\pi \sigma^2}} \exp{\left(- \frac{1}{2 \sigma^2} (x - \mu)^2\right)} $$

    Gaussian Dist.에서 모수는 $\mu, \sigma$ 이다.  
    우리가 가진 training data를 가지고 $\mu, \sigma$를 추정하면 결국 p로 추정되는 q를 구할 수 있는 것이다.  
    
    __그렇다면 $\mu, \sigma$를 어떻게 추정하는가?__  
    크게 ML(Maximum Likelihood) 방법과 MAP(Maximum A Posteriori) 방법이 있지만 여기서는 ML 방법만 보자.  
    그전에 ML과 MAP에 대해 간단히 설명하면,  
    Bayes' rule에서 likelihood 부분만 maximize 시키는 것이 ML이고, likelihood 뿐만 아니라 prior까지 사용하는 것이 MAP이다.   
    <br \>
        - __Bayes' rule__
    $$ P(m|\mathbf{x}) = \frac{p(\mathbf{x}|m)P(m)}{p(\mathbf{x})} $$ 
$ P(m|\mathbf{x}) $ : posterior  
$ p(\mathbf{x}|m) $ : likelihood    
$ P(m) $ : prior  

    likelihood는 training data가 충분하지 않을 때 굉장히 부정확할 수 있는데, MAP는 이를 연구자가 사전에 알고 있는 지식(prior)로 보정해줄 수 있는 것이다.  
    그러나 training data가 충분히 많다면 ML이나 MAP나 비슷한 값을 리턴하게 된다.  
    
    Likelihood function을 구하는 식은 다음과 같다.  
    $$ \begin{matrix}
    L(X;\theta) = p(x_0, ..., x_{N-1}; \theta) = \prod_{i=0}^{N}p(x_i;\theta) \\
                                               = \frac{1}{(2\pi\sigma^2)^{N|2}} \exp{\left(-\frac{1}{2\sigma^2} \sum_{i=0}^{N} (x_{i}-\theta)^2\right)}
       \end{matrix}$$  
       
    Likelihood function을 maximize하는 theta 값($ \underset{\theta}{argmax} L(x;\theta) $ 일때 $ \theta $)를 구하면 된다.  
    구해보면 
    $$ \hat{\mu} = \frac{1}{N} \sum_{t=0}^{N-1} X_t $$ 
    $$ \hat{\sigma}^2 = \frac{1}{N} \sum_{t=0}^{N-1} (X_t - \hat{\mu})^2 $$
    이렇게 나오게 된다.  
    <br \>
    
    __c.f.) cdf (cdf는 확률이다.)__  
    Consider a real valued random variable X
        - Cumulative distribution function(cdf) F(x) for X :
    $$ F(x) = P(X \le x) $$
        - To obtain the probability of falling in an interval we can do the following :
    $$ \begin{matrix}
        P(a < X \le b) = P(X \le b) - P(X \le a) \\
                       = F(b) - F(a) 
       \end{matrix} $$  
    <br \>  
    
    __c.f.) pdf (pdf는 확률이 아니다. 따라서 Gaussian Dist.도 확률이 아니다.)__  
       - The rate of change of the cdf gives us the probability density function(pdf), p(x):
       $$ p(x) = \frac{dx}{d}F(x) = F'(x) $$
       $$ F(x) = \int_{-\infty}^{x} p(x)dx $$
       - p(x) is __not__ the probability that X has value x. But the pdf is proportional to the probability that X lies in a samll interval centred on x.
       - Notation: p for pdf, P for porbability  
    
    
- __GMM__  
mixture model은 다음과 같다.  
$$ p(\mathbf{x}) = \sum_{m=1}^{M} p(\mathbf{x}|m)P(m) $$
    

In [ ]:
$$ p(x|\theta) = \frac{1}{\root{2\pi \sigma^2}} $$
\exp{-\frac{1}{2\sigma^2(x - \mu)^2}